In [1]:
%matplotlib inline

import xarray as xr
import numpy as np

from tonic.io import read_configobj

In [7]:
# ================================================ #
# Command line arguments
# ================================================ #
cfg = read_configobj(('cfg/prepare_state_perturbation.ArkRed.sm_pert_0.5.cfg'))

In [8]:
# ================================================ #
# Extract dimension info
# ================================================ #
# --- Load VIC param file --- #
ds_param = xr.open_dataset(cfg['VIC']['vic_param_nc'])
# --- Extract dimension info needed --- #
lat_coord = ds_param['lat'].values
lon_coord = ds_param['lon'].values
nlayer_coord = ds_param['nlayer'].values

In [9]:
# ================================================ #
# Initialize state perturbation DataArray
# ================================================ #
da_scale = xr.DataArray(np.zeros([len(nlayer_coord), len(lat_coord), len(lon_coord)]),
                        dims=['nlayer', 'lat', 'lon'],
                        coords=[nlayer_coord, lat_coord, lon_coord])

In [11]:
# ================================================ #
# Calculate and/or put perturbation amount into da_scale
# The resulting values in da_scale is the standard deviation of perturbation magnitude in [mm]
# ================================================ #
if cfg['PERTURB']['method'] == 'constant':
    list_state_pert_std = cfg['PERTURB']['param']  # one value for each layer
    for i in range(len(nlayer_coord)):
        da_scale[i, :, :] = list_state_pert_std[i]
# Add unit attribute
da_scale.attrs['unit'] = 'mm'

In [13]:
# ================================================ #
# Save da_scale to file
# ================================================ #
ds_scale = xr.Dataset({'sm_pert_scale': da_scale})
ds_scale.to_netcdf(cfg['OUTPUT']['scale_nc'], format='NETCDF4_CLASSIC')